In [ ]:
from bs4 import BeautifulSoup
import requests
import zipfile

In [ ]:
dominio = 'https://www.gov.br'
URL = f'{dominio}/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos'
# Em caso de futura necessidade de alteração, a URL pode ser alterada para uma variável, sendo assim o foi deixado o dominio e o URL separados.
headers = {'User-Agent': 'Mozilla/5.0'}
pagina = requests.get(URL, headers = headers)

In [ ]:
# Para garantir que o site esteja no ar iremos realizar através do if que a página está em funcionamento, caso esteja, ou seja, caso o código impresso seja 200.
if pagina.status_code == 200:
    # A partir do BeautifulSoup vamos realizar o parsing do HTML para que em seguida possamos buscar os links para download dos arquivos solicitados em PDF.
    soup = BeautifulSoup(pagina.text, 'html.parser')
    # Devido a necessidade de compactar o arquivo vou seguir criando uma lista para armazenar os dados dos arquivos baixados
    arquivos_baixados = []

    # Iremos buscar ambos os links agora.
    link_anexo1 = soup.find('a', string="Anexo I.")
    if link_anexo1:
        link_anexo1_url = link_anexo1.get('href')
        # Após checar se o mesmo foi encontrado, iremos realizar o download do arquivo PDF do Anexo I.
        response = requests.get(link_anexo1_url)
        if response.status_code == 200:
            # Ao invés de salvar o arquivo no disco, vamos adicionar diretamente ao arquivo ZIP
            arquivos_baixados.append(("Anexo_I.pdf", response.content))
            print("Anexo I foi devidamente instalado e será armazenado no arquivo ZIP.")
        else:
            print(f"Falha ao baixar o Anexo I. Status: {response.status_code}")
    else:
        print("Link do Anexo I não encontrado.")

    # Usamos como parâmetro para localizar ambos os arquivos a classe "a" conforme orientado em seu código HTML, juntamente com o nome do próprio arquivo, afim de encontrar o arquivo em questão.
    link_anexo2 = soup.find('a', string="Anexo II.")
    if link_anexo2:
        link_anexo2_url = link_anexo2.get('href')
        # Assim como no anexo I seguimos realizando a mesma ação no anexo II, realizando o download do arquivo PDF.
        response = requests.get(link_anexo2_url)
        if response.status_code == 200:
            # Adicionar também ao arquivo ZIP
            arquivos_baixados.append(("Anexo_II.pdf", response.content))
            print("Anexo II foi devidamente instalado e será armazenado no arquivo ZIP.")
        else:
            print(f"Falha ao baixar o Anexo II. Status: {response.status_code}")
    else:
        print("Link do Anexo II não encontrado.")
    
    # Após listar todos os arquivos, iremos compactá-los em um único arquivo ZIP.
    if arquivos_baixados:
        with zipfile.ZipFile("Anexos.zip", "w") as zipf:
            for nome_arquivo, conteudo in arquivos_baixados:
                zipf.writestr(nome_arquivo, conteudo)  # Salvando diretamente no ZIP
        print("Os Arquivos foram compactados com sucesso em 'Anexos.zip'.")
else:
    print(f"Falha ao acessar a página. Status: {pagina.status_code}") 
